## Extraer Nombres

In [ ]:
from pathlib import Path

# === Ruta base donde buscar ===
carpeta_base = Path("20251003/20251003")

# === Buscar todos los archivos cuyo nombre empiece por 'ADP_DTM_' ===
archivos_encontrados = list(carpeta_base.glob("ADP_DTM_*"))

# === Mostrar resultados ===
if archivos_encontrados:
    print("[DEBUG] Archivos encontrados:")
    for archivo in archivos_encontrados:
        print("-", archivo.name)
else:
    print("[DEBUG] No se encontraron archivos que empiecen por 'ADP_DTM_'.")


In [ ]:
import pandas as pd
from pathlib import Path

# === Ruta base donde buscar ===
carpeta_base = Path("20251003/20251003")

# === Buscar todos los archivos que empiecen por 'ADP_DTM_' ===
archivos_csv = list(carpeta_base.glob("ADP_DTM_*.csv"))

print(f"[DEBUG] {len(archivos_csv)} archivos encontrados.")

# === Función para detectar variables de fecha ===
def detectar_columnas_fecha(nombre_archivo: Path) -> list[str]:
    try:
        # Leer solo la primera fila (encabezados)
        encabezados = pd.read_csv(nombre_archivo, nrows=0).columns
        # Detectar columnas con palabras clave relacionadas a fecha
        columnas_fecha = [
            col for col in encabezados
            if any(palabra in col.lower() for palabra in ["fecha", "date", "fch"])
        ]
        print(f"[DEBUG] {nombre_archivo.name}: {columnas_fecha}")
        return columnas_fecha
    except Exception as e:
        print(f"[DEBUG] Error leyendo {nombre_archivo.name}: {e}")
        return []

# === Recorrer y registrar resultados ===
resultado = {}
for archivo in archivos_csv:
    columnas = detectar_columnas_fecha(archivo)
    if columnas:
        resultado[archivo.name] = columnas

# === Mostrar resumen ===
print("\n=== Resumen de columnas tipo fecha ===")
for nombre, columnas in resultado.items():
    print(f"{nombre}: {', '.join(columnas)}")


In [ ]:
import pandas as pd
from pathlib import Path

# === Ruta base ===
carpeta_base = Path("20251003/20251003")
archivos_csv = list(carpeta_base.glob("ADP_DTM_*.csv"))

print(f"[DEBUG] {len(archivos_csv)} archivos encontrados.")

def detectar_columnas_fecha(nombre_archivo: Path) -> list[str]:
    """Detecta columnas de fecha según el encabezado."""
    try:
        encabezados = pd.read_csv(nombre_archivo, nrows=0).columns
        columnas_fecha = [
            col for col in encabezados
            if any(p in col.lower() for p in ["fecha", "date", "fch"])
        ]
        return columnas_fecha
    except Exception as e:
        print(f"[DEBUG] Error leyendo encabezados de {nombre_archivo.name}: {e}")
        return []

def contar_por_año(nombre_archivo: Path, columnas_fecha: list[str]) -> pd.DataFrame:
    """Cuenta la frecuencia de registros por año en las columnas de fecha detectadas."""
    try:
        df = pd.read_csv(nombre_archivo, usecols=columnas_fecha, low_memory=False)
        tabla_final = []
        for col in columnas_fecha:
            # Convertir a fecha
            df[col] = pd.to_datetime(df[col], errors="coerce")
            # Contar por año
            conteo = df[col].dt.year.value_counts().sort_index()
            temp = pd.DataFrame({
                "archivo": nombre_archivo.name,
                "columna": col,
                "año": conteo.index,
                "frecuencia": conteo.values
            })
            tabla_final.append(temp)
        return pd.concat(tabla_final, ignore_index=True)
    except Exception as e:
        print(f"[DEBUG] Error procesando {nombre_archivo.name}: {e}")
        return pd.DataFrame(columns=["archivo", "columna", "año", "frecuencia"])

# === Procesar todos los archivos ===
todas_las_tablas = []
for archivo in archivos_csv:
    columnas_fecha = detectar_columnas_fecha(archivo)
    if columnas_fecha:
        print(f"[DEBUG] {archivo.name}: columnas de fecha -> {columnas_fecha}")
        tabla = contar_por_año(archivo, columnas_fecha)
        if not tabla.empty:
            todas_las_tablas.append(tabla)

# === Unir resultados ===
if todas_las_tablas:
    resumen = pd.concat(todas_las_tablas, ignore_index=True)
    print("\n=== Tabla de frecuencias por año ===")
    print(resumen)
    # Guardar opcionalmente
    resumen.to_csv("frecuencias_fechas_ADP_DTM.csv", index=False)
    print("\n[DEBUG] Guardado como frecuencias_fechas_ADP_DTM.csv")
else:
    print("[DEBUG] No se generó ninguna tabla de frecuencias.")


In [ ]:
import pandas as pd
from pathlib import Path

# === Ruta base ===
carpeta_base = Path("20251003/20251003")
archivos_csv = list(carpeta_base.glob("ADP_DTM_*.csv"))

print(f"[DEBUG] {len(archivos_csv)} archivos encontrados.")

def detectar_columnas_fecha(nombre_archivo: Path) -> list[str]:
    """Detecta columnas de fecha según el encabezado."""
    try:
        encabezados = pd.read_csv(nombre_archivo, nrows=0).columns
        columnas_fecha = [
            col for col in encabezados
            if any(p in col.lower() for p in ["fecha", "date", "fch"])
        ]
        return columnas_fecha
    except Exception as e:
        print(f"[DEBUG] Error leyendo encabezados de {nombre_archivo.name}: {e}")
        return []

def contar_por_proyecto(nombre_archivo: Path, columnas_fecha: list[str]) -> pd.DataFrame:
    """Cuenta la frecuencia de registros por SkIdProyecto y columnas de fecha."""
    try:
        # Verificar si la columna SkIdProyecto existe
        encabezados = pd.read_csv(nombre_archivo, nrows=0).columns
        if "SkIdProyecto" not in encabezados:
            print(f"[DEBUG] {nombre_archivo.name}: sin columna 'SkIdProyecto', omitido.")
            return pd.DataFrame(columns=["archivo", "columna", "SkIdProyecto", "frecuencia"])
        
        columnas_a_leer = ["SkIdProyecto"] + columnas_fecha
        df = pd.read_csv(nombre_archivo, usecols=columnas_a_leer, low_memory=False)
        tabla_final = []
        
        for col in columnas_fecha:
            df[col] = pd.to_datetime(df[col], errors="coerce")
            conteo = df.groupby("SkIdProyecto")[col].count().reset_index()
            conteo.rename(columns={col: "frecuencia"}, inplace=True)
            conteo["archivo"] = nombre_archivo.name
            conteo["columna"] = col
            tabla_final.append(conteo)

        return pd.concat(tabla_final, ignore_index=True)
    except Exception as e:
        print(f"[DEBUG] Error procesando {nombre_archivo.name}: {e}")
        return pd.DataFrame(columns=["archivo", "columna", "SkIdProyecto", "frecuencia"])

# === Procesar todos los archivos ===
todas_las_tablas = []
for archivo in archivos_csv:
    columnas_fecha = detectar_columnas_fecha(archivo)
    if columnas_fecha:
        print(f"[DEBUG] {archivo.name}: columnas de fecha -> {columnas_fecha}")
        tabla = contar_por_proyecto(archivo, columnas_fecha)
        if not tabla.empty:
            todas_las_tablas.append(tabla)

# === Unir resultados ===
if todas_las_tablas:
    resumen = pd.concat(todas_las_tablas, ignore_index=True)
    print("\n=== Tabla de frecuencias por SkIdProyecto ===")
    print(resumen)
    resumen.to_csv("frecuencias_por_SkIdProyecto_ADP_DTM.csv", index=False)
    print("\n[DEBUG] Guardado como frecuencias_por_SkIdProyecto_ADP_DTM.csv")
else:
    print("[DEBUG] No se generó ninguna tabla de frecuencias.")


In [ ]:
import pandas as pd
from pathlib import Path

# === Rutas base ===
carpeta_base = Path("20251003/20251003")
archivos_csv = list(carpeta_base.glob("ADP_DTM_*.csv"))

print(f"[DEBUG] {len(archivos_csv)} archivos encontrados.")

# === Cargar tabla Proyecto ===
ruta_proyecto = carpeta_base / "ADP_DTM_DIM.Proyecto.csv"
try:
    df_proyecto = pd.read_csv(ruta_proyecto, low_memory=False)
    columnas_existentes = df_proyecto.columns
    posibles_nombres = [c for c in columnas_existentes if any(p in c.lower() for p in ["macro", "nombre", "proyecto"])]
    print(f"[DEBUG] Columnas disponibles en Proyecto: {posibles_nombres}")
except Exception as e:
    print(f"[DEBUG] No se pudo leer ADP_DTM_DIM.Proyecto.csv: {e}")
    df_proyecto = pd.DataFrame(columns=["SkIdProyecto", "MacroProyecto Descripcion", "Nombre Proyecto"])

# === Funciones auxiliares ===
def detectar_columnas_fecha(nombre_archivo: Path) -> list[str]:
    try:
        encabezados = pd.read_csv(nombre_archivo, nrows=0).columns
        columnas_fecha = [
            col for col in encabezados
            if any(p in col.lower() for p in ["fecha", "date", "fch"])
        ]
        return columnas_fecha
    except Exception as e:
        print(f"[DEBUG] Error leyendo encabezados de {nombre_archivo.name}: {e}")
        return []

def contar_por_proyecto(nombre_archivo: Path, columnas_fecha: list[str]) -> pd.DataFrame:
    try:
        encabezados = pd.read_csv(nombre_archivo, nrows=0).columns
        if "SkIdProyecto" not in encabezados:
            print(f"[DEBUG] {nombre_archivo.name}: sin columna 'SkIdProyecto', omitido.")
            return pd.DataFrame(columns=["archivo", "columna", "SkIdProyecto", "frecuencia"])

        columnas_a_leer = ["SkIdProyecto"] + columnas_fecha
        df = pd.read_csv(nombre_archivo, usecols=columnas_a_leer, low_memory=False)
        tabla_final = []

        for col in columnas_fecha:
            df[col] = pd.to_datetime(df[col], errors="coerce")
            conteo = df.groupby("SkIdProyecto")[col].count().reset_index()
            conteo.rename(columns={col: "frecuencia"}, inplace=True)
            conteo["archivo"] = nombre_archivo.name
            conteo["columna"] = col
            tabla_final.append(conteo)

        return pd.concat(tabla_final, ignore_index=True)
    except Exception as e:
        print(f"[DEBUG] Error procesando {nombre_archivo.name}: {e}")
        return pd.DataFrame(columns=["archivo", "columna", "SkIdProyecto", "frecuencia"])

# === Procesar todos los archivos ===
todas_las_tablas = []
for archivo in archivos_csv:
    columnas_fecha = detectar_columnas_fecha(archivo)
    if columnas_fecha:
        print(f"[DEBUG] {archivo.name}: columnas de fecha -> {columnas_fecha}")
        tabla = contar_por_proyecto(archivo, columnas_fecha)
        if not tabla.empty:
            todas_las_tablas.append(tabla)

# === Combinar resultados y unir con Proyecto ===
if todas_las_tablas:
    resumen = pd.concat(todas_las_tablas, ignore_index=True)
    print(f"[DEBUG] Total filas antes del merge: {len(resumen)}")

    # Asegurar consistencia de nombre de llave
    if "SkIdProyecto" not in df_proyecto.columns:
        print("[DEBUG] Advertencia: no se encontró 'SkIdProyecto' en la tabla Proyecto.")
    else:
        resumen = resumen.merge(
            df_proyecto[["SkIdProyecto", "MacroProyecto", "Nombre Proyecto"]],
            on="SkIdProyecto",
            how="left"
        )
    
    # Guardar resultado final
    resumen.to_csv("frecuencias_por_SkIdProyecto_completo.csv", index=False)
    print("\n=== Resumen final ===")
    print(resumen.head(20))
    print("\n[DEBUG] Guardado como frecuencias_por_SkIdProyecto_completo.csv")
else:
    print("[DEBUG] No se generó ninguna tabla de frecuencias.")


# Exploracion Imprevistos

## v1

In [ ]:
import pandas as pd
from pathlib import Path

# === Ruta base ===
carpeta_base = Path("20251003/20251003")
archivos_csv = list(carpeta_base.glob("ADP_DTM_*.csv"))

In [5]:

# Filtrar todos los que contienen 'Pedidos'
archivos_pedidos = [a for a in archivos_csv if "Pedidos" in a.name]

print("[DEBUG] Archivos encontrados con 'Pedidos':")
for i, archivo in enumerate(archivos_pedidos, start=1):
    print(f"  [{i}] {archivo.name}")

[DEBUG] Archivos encontrados con 'Pedidos':
  [1] ADP_DTM_DIM.EspecicficacionDePedidos.csv
  [2] ADP_DTM_FACT.AuditoriaPedidos.csv
  [3] ADP_DTM_FACT.Pedidos.csv


In [7]:
[print(f"\n[DEBUG] {a.name} → {list(pd.read_csv(a, nrows=0).columns)}") for a in archivos_pedidos]



[DEBUG] ADP_DTM_DIM.EspecicficacionDePedidos.csv → ['SkIdEmpresa', 'SkIdPedido', 'Codigo Orden De Compra', 'Pedido Urgente', 'Tipo Pedido']

[DEBUG] ADP_DTM_FACT.AuditoriaPedidos.csv → ['SkIdEmpresa', 'SkIdProyecto', 'SkIdPedido', 'SkIdInsumo', 'SkIdFecha', 'SkIdUsuario', 'SkIdEstado', 'EventoPedido']

[DEBUG] ADP_DTM_FACT.Pedidos.csv → ['SkIdEmpresa', 'SkIdProyecto', 'SkIdCapitulo', 'SkIdPedido', 'SkIdItems', 'SkIdInsumo', 'SkIdFechaPedido', 'SkIdFechaRequerido', 'SkIdEstado', 'Cantidad']


[None, None, None]

In [ ]:
import pandas as pd
from pathlib import Path

def build_percentage_table_by_project():
    print("[DEBUG] Starting percentage table build...")

    base_folder = Path("20251003/20251003")
    file_specification = base_folder / "ADP_DTM_DIM.EspecicficacionDePedidos.csv"
    file_orders = base_folder / "ADP_DTM_FACT.Pedidos.csv"

    print(f"[DEBUG] Reading: {file_specification.name}")
    dataframe_specification = pd.read_csv(file_specification)

    print(f"[DEBUG] Reading: {file_orders.name}")
    dataframe_orders = pd.read_csv(file_orders)

    required_columns_spec = ["SkIdEmpresa", "SkIdPedido", "Tipo Pedido"]
    required_columns_orders = ["SkIdEmpresa", "SkIdProyecto", "SkIdPedido"]

    dataframe_specification = dataframe_specification[required_columns_spec]
    dataframe_orders = dataframe_orders[required_columns_orders]

    print("[DEBUG] Merging specification with orders on SkIdEmpresa + SkIdPedido...")
    dataframe_merged = dataframe_orders.merge(
        dataframe_specification,
        on=["SkIdEmpresa", "SkIdPedido"],
        how="left"
    )

    print("[DEBUG] Filling missing 'Tipo Pedido' with 'NA'...")
    dataframe_merged["Tipo Pedido"] = dataframe_merged["Tipo Pedido"].fillna("NA")

    print("[DEBUG] Computing percentage distribution by project...")
    # One-table output: projects as rows, Tipo Pedido as columns, values are percentages
    percentage_table = (
        dataframe_merged
        .groupby(["SkIdProyecto", "Tipo Pedido"], dropna=False)
        .size()
        .groupby(level=0)
        .apply(lambda series: series / series.sum() * 100.0)
        .unstack("Tipo Pedido", fill_value=0.0)
        .round(2)
        .sort_index()
    )

    print("[DEBUG] Done. Preview:")
    print(percentage_table.to_string())
    return percentage_table

if __name__ == "__main__":
    _ = build_percentage_table_by_project()


In [27]:
import pandas as pd
from pathlib import Path

carpeta = Path("20251003/20251003")
df_spec = pd.read_csv(carpeta / "ADP_DTM_DIM.EspecicficacionDePedidos.csv")
df_ped = pd.read_csv(carpeta / "ADP_DTM_FACT.Pedidos.csv")

df = df_ped[["SkIdEmpresa", "SkIdProyecto", "SkIdPedido"]].merge(
    df_spec[["SkIdEmpresa", "SkIdPedido", "Tipo Pedido"]],
    on=["SkIdEmpresa", "SkIdPedido"], how="left"
)

df["Tipo Pedido"] = df["Tipo Pedido"].fillna("NA")

tabla = (
    df.groupby(["SkIdProyecto", "Tipo Pedido"])
      .size()
      .groupby(level=0)
      .apply(lambda x: round(100 * x / x.sum(), 2))
      .unstack(fill_value=0)
      .sort_index()
)

display(tabla)


,Tipo Pedido,ADICIONAL,PRESUPUESTAL
SkIdProyecto,SkIdProyecto,,
1006,1006,100.00,0.00
1009,1009,100.00,0.00
10021,10021,100.00,0.00
10029,10029,100.00,0.00
10030,10030,100.00,0.00
10031,10031,98.07,1.93
10035,10035,100.00,0.00
100116,100116,100.00,0.00
100118,100118,100.00,0.00


In [23]:

# Option 2 — moderate view (e.g., 200 rows)
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 50)

In [38]:
import pandas as pd
from pathlib import Path
from IPython.display import display

carpeta = Path("20251003/20251003")
df_spec = pd.read_csv(carpeta / "ADP_DTM_DIM.EspecicficacionDePedidos.csv")
df_ped  = pd.read_csv(carpeta / "ADP_DTM_FACT.Pedidos.csv")
df_cap  = pd.read_csv(carpeta / "ADP_DTM_DIM.CapituloPresupuesto.csv")

# Merge Pedido + Tipo Pedido
df = df_ped[["SkIdEmpresa", "SkIdCapitulo", "SkIdPedido"]].merge(
    df_spec[["SkIdEmpresa", "SkIdPedido", "Tipo Pedido"]],
    on=["SkIdEmpresa", "SkIdPedido"], how="left"
)
df["Tipo Pedido"] = df["Tipo Pedido"].fillna("NA")

# Percentages by capítulo (rows sum to 100)
tabla_pct = pd.crosstab(df["SkIdCapitulo"], df["Tipo Pedido"], normalize="index") * 100
tabla_pct = tabla_pct.round(2)

# Total count per capítulo
tabla_count = df.groupby("SkIdCapitulo").size().rename("Total Pedidos")

# Combine into one table
tabla = tabla_pct.merge(tabla_count, on="SkIdCapitulo").reset_index()

# Add capítulo description
df_cap = df_cap[["SkIdCapitulo", "Capitulo Descripcion"]].drop_duplicates()
tabla = tabla.merge(df_cap, on="SkIdCapitulo", how="left")

# Reorder columns
tipo_cols = [c for c in tabla.columns if c not in ["SkIdCapitulo", "Capitulo Descripcion", "Total Pedidos"]]
tabla = tabla[["SkIdCapitulo", "Capitulo Descripcion"] + tipo_cols + ["Total Pedidos"]]

# === Ordenar primero por 'Adicional' y luego por 'Total Pedidos' ===
col_adic = next((c for c in tipo_cols if c.lower() == "adicional"), None)

if col_adic:
    tabla = tabla.sort_values([col_adic, "Total Pedidos"], ascending=[False, False])
else:
    tabla = tabla.sort_values("Total Pedidos", ascending=False)

display(tabla)


display(tabla)


,SkIdCapitulo,Capitulo Descripcion,ADICIONAL,PRESUPUESTAL,Total Pedidos
264,1002112832,INSTALACIONES HIDROSANITARIAS,100.0,0.0,1575
340,1002263101,INSTALACIONES HIDROSANITARIAS,100.0,0.0,1210
410,1002393392,INSTALACIONES ELECTRICAS,100.0,0.0,1203
389,1002373474,GASTOS GENERALES,100.0,0.0,1151
235,1002042740,GASTOS GENERALES,100.0,0.0,1056
...,...,...,...,...,...
509,1002534237,EXTERIORES,0.0,100.0,2
479,1002503151,CUBIERTAS Y CIELORASOS,0.0,100.0,1
491,1002503166,"CERRADURAS, ESPEJOS Y NOMENCLATURA",0.0,100.0,1
567,1002694051,REMATES Y ASEO,0.0,100.0,1


,SkIdCapitulo,Capitulo Descripcion,ADICIONAL,PRESUPUESTAL,Total Pedidos
264,1002112832,INSTALACIONES HIDROSANITARIAS,100.0,0.0,1575
340,1002263101,INSTALACIONES HIDROSANITARIAS,100.0,0.0,1210
410,1002393392,INSTALACIONES ELECTRICAS,100.0,0.0,1203
389,1002373474,GASTOS GENERALES,100.0,0.0,1151
235,1002042740,GASTOS GENERALES,100.0,0.0,1056
...,...,...,...,...,...
509,1002534237,EXTERIORES,0.0,100.0,2
479,1002503151,CUBIERTAS Y CIELORASOS,0.0,100.0,1
491,1002503166,"CERRADURAS, ESPEJOS Y NOMENCLATURA",0.0,100.0,1
567,1002694051,REMATES Y ASEO,0.0,100.0,1


## crossmap

In [2]:
import pandas as pd
from pathlib import Path
from IPython.display import display

carpeta = Path("20251003/20251003")
df_spec = pd.read_csv(carpeta / "ADP_DTM_DIM.EspecicficacionDePedidos.csv")    # ... 'Tipo Pedido'
df_ped  = pd.read_csv(carpeta / "ADP_DTM_FACT.Pedidos.csv")                    # ... 'SkIdCapitulo', 'SkIdProyecto'
df_cap  = pd.read_csv(carpeta / "ADP_DTM_DIM.CapituloPresupuesto.csv")         # ... 'Capitulo Descripcion'

print("[DEBUG] Merging 'Tipo Pedido' into Pedidos...")
df = df_ped[["SkIdEmpresa", "SkIdCapitulo", "SkIdProyecto", "SkIdPedido"]].merge(
    df_spec[["SkIdEmpresa", "SkIdPedido", "Tipo Pedido"]],
    on=["SkIdEmpresa", "SkIdPedido"], how="left"
)

df["Tipo Pedido"] = df["Tipo Pedido"].fillna("NA")
target_tipo = "Presupuestal"   # <- change to "Adicional" if you prefer that view

print(f"[DEBUG] Building Capítulo × Proyecto crossmap for '{target_tipo}' (%) ...")
crossmap = (
    df.groupby(["SkIdCapitulo", "SkIdProyecto"])["Tipo Pedido"]
      .apply(lambda s: (s == target_tipo).mean() * 100)
      .unstack(fill_value=0.0)
      .round(2)
      .reset_index()
)

print("[DEBUG] Attaching 'Capitulo Descripcion'...")
df_cap = df_cap[["SkIdCapitulo", "Capitulo Descripcion"]].drop_duplicates()
crossmap = crossmap.merge(df_cap, on="SkIdCapitulo", how="left")

# Put the name near the id, keep projects as columns
cols_fixed = ["SkIdCapitulo", "Capitulo Descripcion"]
cols_proj = [c for c in crossmap.columns if c not in cols_fixed]
crossmap = crossmap[cols_fixed + cols_proj]

display(crossmap)


[DEBUG] Merging 'Tipo Pedido' into Pedidos...
[DEBUG] Building Capítulo × Proyecto crossmap for 'Presupuestal' (%) ...
[DEBUG] Attaching 'Capitulo Descripcion'...


,SkIdCapitulo,Capitulo Descripcion,1006,1009,10021,10029,10030,10031,10035,100116,...,100262,100264,100268,100269,100275,100277,100278,100283,100294,100295
0,1006166,INSTALACIONES HIDROSANITARIAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1009342,GASTOS GENERALES,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10021454,POSTVENTAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10030986,IMPREVISTOS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100291591,VIAS Y ANDENES,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,1002834088,SALA DE VENTAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
592,1002834102,APARTAMENTO MODELO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
593,1002944493,CIMENTACION,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
594,1002944494,ESTRUCTURA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import pandas as pd
import altair as alt
from pathlib import Path

carpeta = Path("20251003/20251003")
df_spec = pd.read_csv(carpeta / "ADP_DTM_DIM.EspecicficacionDePedidos.csv")
df_ped  = pd.read_csv(carpeta / "ADP_DTM_FACT.Pedidos.csv")
df_cap  = pd.read_csv(carpeta / "ADP_DTM_DIM.CapituloPresupuesto.csv")

# --- Merge and prepare ---
df = df_ped[["SkIdEmpresa", "SkIdCapitulo", "SkIdProyecto", "SkIdPedido"]].merge(
    df_spec[["SkIdEmpresa", "SkIdPedido", "Tipo Pedido"]],
    on=["SkIdEmpresa", "SkIdPedido"], how="left"
)
df["Tipo Pedido"] = df["Tipo Pedido"].fillna("NA")

target_tipo = "Presupuestal"  # choose one: "Presupuestal" or "Adicional"

crossmap = (
    df.groupby(["SkIdCapitulo", "SkIdProyecto"])["Tipo Pedido"]
      .apply(lambda s: (s == target_tipo).mean() * 100)
      .reset_index(name="Porcentaje")
)

crossmap = crossmap.merge(
    df_cap[["SkIdCapitulo", "Capitulo Descripcion"]].drop_duplicates(),
    on="SkIdCapitulo", how="left"
)

# --- Plot ---
chart = (
    alt.Chart(crossmap)
    .mark_rect()
    .encode(
        x=alt.X("SkIdProyecto:O", title="Proyecto"),
        y=alt.Y("Capitulo Descripcion:O", title="Capítulo"),
        color=alt.Color("Porcentaje:Q", scale=alt.Scale(scheme="blues"), title=f"% {target_tipo}"),
        tooltip=[
            alt.Tooltip("Capitulo Descripcion:N", title="Capítulo"),
            alt.Tooltip("SkIdProyecto:N", title="Proyecto"),
            alt.Tooltip("Porcentaje:Q", title=f"% {target_tipo}", format=".2f"),
        ],
    )
    .properties(width=800, height=500)
)

chart


In [ ]:
import pandas as pd
import altair as alt
from pathlib import Path

carpeta = Path("20251003/20251003")
df_spec = pd.read_csv(carpeta / "ADP_DTM_DIM.EspecicficacionDePedidos.csv")
df_ped  = pd.read_csv(carpeta / "ADP_DTM_FACT.Pedidos.csv")
df_cap  = pd.read_csv(carpeta / "ADP_DTM_DIM.CapituloPresupuesto.csv")
df_proj = pd.read_csv(carpeta / "ADP_DTM_DIM.Proyecto.csv")   # contiene 'Nombre Proyecto'

# --- Preparar ---
df = df_ped[["SkIdEmpresa", "SkIdCapitulo", "SkIdProyecto", "SkIdPedido"]].merge(
    df_spec[["SkIdEmpresa", "SkIdPedido", "Tipo Pedido"]],
    on=["SkIdEmpresa", "SkIdPedido"], how="left"
)
df["Tipo Pedido"] = df["Tipo Pedido"].fillna("NA")

target_tipo = "Presupuestal"  # o "Adicional"

# --- Construir matriz Capítulo × Proyecto ---
crossmap = (
    df.groupby(["SkIdCapitulo", "SkIdProyecto"])["Tipo Pedido"]
      .apply(lambda s: (s == target_tipo).mean() * 100)
      .reset_index(name="Porcentaje")
)

# --- Agregar nombres de Capítulo y Proyecto ---
df_cap = df_cap[["SkIdCapitulo", "Capitulo Descripcion"]].drop_duplicates()
df_proj = df_proj[["SkIdProyecto", "Nombre Proyecto"]].drop_duplicates()

crossmap = (
    crossmap
    .merge(df_cap, on="SkIdCapitulo", how="left")
    .merge(df_proj, on="SkIdProyecto", how="left")
)

# --- Gráfico ---
chart = (
    alt.Chart(crossmap)
    .mark_rect()
    .encode(
        x=alt.X("Nombre Proyecto:O", title="Proyecto"),
        y=alt.Y("Capitulo Descripcion:O", title="Capítulo"),
        color=alt.Color("Porcentaje:Q", scale=alt.Scale(scheme="blues"), title=f"% {target_tipo}"),
        tooltip=[
            alt.Tooltip("Capitulo Descripcion:N", title="Capítulo"),
            alt.Tooltip("Nombre Proyecto:N", title="Proyecto"),
            alt.Tooltip("Porcentaje:Q", title=f"% {target_tipo}", format=".2f"),
        ],
    )
    .properties(width=800, height=500)
)

chart


In [ ]:
import altair as alt
import numpy as np
import pandas as pd

# --- recompute bins safely ---
valid = crossmap_full.loc[crossmap_full["Capitulo Descripcion"] != "TOTAL", "Porcentaje"]
edges = np.unique(np.percentile(valid, [0, 25, 50, 75, 100])).tolist()
labels = [f"{edges[i]:.1f}–{edges[i+1]:.1f}" for i in range(len(edges)-1)]
crossmap_full["Bin Adicional"] = pd.cut(
    crossmap_full["Porcentaje"], bins=edges,
    labels=labels, include_lowest=True, duplicates="drop"
)

crossmap_full["is_total"] = np.where(
    (crossmap_full["Capitulo Descripcion"] == "TOTAL") |
    (crossmap_full["Nombre Proyecto"] == "TOTAL"), 1, 0
)

# --- palettes ---
reds = ["#fff5eb", "#fcbba1", "#fb6a4a", "#a50f15"]   # normal cells
blues = ["#deebf7", "#9ecae1", "#3182bd", "#08519c"]  # totals
# --- Replace the reds scale with a distinct categorical palette ---
# (4 distinct, color-blind-safe choices)
palette_distinct = ["#1b9e77", "#d95f02", "#7570b3", "#e7298a"]

chart = (
    alt.Chart(crossmap_full)
    .mark_rect(stroke="black")
    .encode(
        x=alt.X("Nombre Proyecto:O", title="Proyecto"),
        y=alt.Y("Capitulo Descripcion:O", title="Capítulo"),
        color=alt.Color("Bin Adicional:N",
                        sort=labels,
                        scale=alt.Scale(range=palette_distinct),
                        legend=alt.Legend(title="% Adicional (binned)")),
        strokeWidth=alt.StrokeWidth("is_total:Q", scale=alt.Scale(domain=[0,1], range=[0.2,1.5])),
        opacity=alt.Opacity("is_total:Q", scale=alt.Scale(domain=[0,1], range=[0.9,1.0])),
        tooltip=[
            alt.Tooltip("Capitulo Descripcion:N", title="Capítulo"),
            alt.Tooltip("Nombre Proyecto:N", title="Proyecto"),
            alt.Tooltip("Porcentaje:Q", title="% Adicional", format=".2f"),
            alt.Tooltip("Bin Adicional:N", title="Rango"),
        ],
    )
    .properties(width=950, height=650)
)

chart



In [ ]:
import altair as alt
import numpy as np

# --- 1) Refine bins again for better sub-70% detail ---
valid = crossmap_full.loc[crossmap_full["Capitulo Descripcion"] != "TOTAL", "Porcentaje"]
edges = np.unique(np.percentile(valid, [0, 10, 25, 50, 70, 85, 100])).tolist()
labels = [f"{edges[i]:.1f}–{edges[i+1]:.1f}" for i in range(len(edges)-1)]
crossmap_full["Bin Adicional"] = pd.cut(
    crossmap_full["Porcentaje"], bins=edges,
    labels=labels, include_lowest=True, duplicates="drop"
)

# --- 2) Mark totals and sort them last ---
crossmap_full["is_total"] = np.where(
    (crossmap_full["Capitulo Descripcion"] == "TOTAL") |
    (crossmap_full["Nombre Proyecto"] == "TOTAL"), 1, 0
)
crossmap_full = crossmap_full.sort_values(
    by=["is_total", "Capitulo Descripcion", "Nombre Proyecto"], ascending=[True, True, True]
)

# --- 3) Unified categorical palette (distinct colors) ---
palette_distinct = ["#1b9e77", "#d95f02", "#7570b3", "#e7298a", "#66a61e", "#e6ab02"]

# --- 4) Plot (no nested Color condition) ---
chart = (
    alt.Chart(crossmap_full)
    .mark_rect(stroke="black")
    .encode(
        x=alt.X("Nombre Proyecto:O", title="Proyecto"),
        y=alt.Y("Capitulo Descripcion:O", title="Capítulo"),
        color=alt.Color("Bin Adicional:N",
                        sort=labels,
                        scale=alt.Scale(range=palette_distinct),
                        legend=alt.Legend(title="% Adicional (binned)")),
        strokeWidth=alt.StrokeWidth("is_total:Q",
                                    scale=alt.Scale(domain=[0, 1], range=[0.3, 2.5]),
                                    legend=None),
        opacity=alt.Opacity("is_total:Q",
                            scale=alt.Scale(domain=[0, 1], range=[0.9, 1.0]),
                            legend=None),
        tooltip=[
            alt.Tooltip("Capitulo Descripcion:N", title="Capítulo"),
            alt.Tooltip("Nombre Proyecto:N", title="Proyecto"),
            alt.Tooltip("Porcentaje:Q", title="% Adicional", format=".2f"),
            alt.Tooltip("Bin Adicional:N", title="Rango"),
        ],
    )
    .properties(width=950, height=650)
)

chart


In [69]:
import sys, shlex, subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "vl-convert-python>=1.6.0"])


0

In [ ]:
# === 0) Ensure full Capítulo–Proyecto grid (no white gaps) ===
all_cap = crossmap_full["Capitulo Descripcion"].unique()
all_proj = crossmap_full["Nombre Proyecto"].unique()

# create full cartesian product
full_grid = pd.MultiIndex.from_product(
    [all_cap, all_proj],
    names=["Capitulo Descripcion", "Nombre Proyecto"]
).to_frame(index=False)

# merge so every combination exists; fill missing Porcentaje with 0
crossmap_full = (
    full_grid.merge(crossmap_full, on=["Capitulo Descripcion", "Nombre Proyecto"], how="left")
    .fillna({"Porcentaje": 0, "is_total": 0})
)

# re-compute bin labels for the filled data
crossmap_full["Bin Adicional"] = pd.cut(
    crossmap_full["Porcentaje"],
    bins=edges,
    labels=labels,
    include_lowest=True,
    duplicates="drop"
)

# === 1) Force TOTAL rows/columns to appear last ===
cat_cap = [c for c in all_cap if c != "TOTAL"] + ["TOTAL"]
cat_proj = [p for p in all_proj if p != "TOTAL"] + ["TOTAL"]

crossmap_full["Capitulo Descripcion"] = pd.Categorical(
    crossmap_full["Capitulo Descripcion"], categories=cat_cap, ordered=True)
crossmap_full["Nombre Proyecto"] = pd.Categorical(
    crossmap_full["Nombre Proyecto"], categories=cat_proj, ordered=True)

# === 2) Re-plot with same palette ===
chart = (
    alt.Chart(crossmap_full)
    .mark_rect(stroke="black")
    .encode(
        x=alt.X("Nombre Proyecto:O", sort=cat_proj, title="Proyecto"),
        y=alt.Y("Capitulo Descripcion:O", sort=cat_cap, title="Capítulo"),
        color=alt.Color("Bin Adicional:N",
                        scale=alt.Scale(range=palette_distinct),
                        legend=alt.Legend(title="% Adicional (binned)")),
        strokeWidth=alt.StrokeWidth("is_total:Q", scale=alt.Scale(domain=[0,1], range=[0.3,2.5]), legend=None),
        opacity=alt.Opacity("is_total:Q", scale=alt.Scale(domain=[0,1], range=[0.9,1.0]), legend=None),
        tooltip=[
            alt.Tooltip("Capitulo Descripcion:N", title="Capítulo"),
            alt.Tooltip("Nombre Proyecto:N", title="Proyecto"),
            alt.Tooltip("Porcentaje:Q", title="% Adicional", format=".2f"),
            alt.Tooltip("Bin Adicional:N", title="Rango"),
        ],
    )
    .properties(width=950, height=650)
)
import altair as alt
alt.data_transformers.enable("vegafusion")  # allows larger dataframes

chart


In [ ]:
import altair as alt

# --- continuous gradient instead of bins ---
crossmap_full["is_total"] = np.where(
    (crossmap_full["Capitulo Descripcion"] == "TOTAL") |
    (crossmap_full["Nombre Proyecto"] == "TOTAL"), 1, 0
)

# Sort totals last for readability
crossmap_full = crossmap_full.sort_values(
    by=["is_total", "Capitulo Descripcion", "Nombre Proyecto"], ascending=[True, True, True]
)

# Continuous color palette (light → dark)
gradient = alt.Scale(scheme="orangered")  # you can also try 'viridis', 'blues', 'inferno'

chart = (
    alt.Chart(crossmap_full)
    .mark_rect(stroke="black")
    .encode(
        x=alt.X("Nombre Proyecto:O", title="Proyecto"),
        y=alt.Y("Capitulo Descripcion:O", title="Capítulo"),
        color=alt.Color("Porcentaje:Q", scale=gradient, title="% Adicional (continuo)"),
        strokeWidth=alt.StrokeWidth("is_total:Q",
                                    scale=alt.Scale(domain=[0, 1], range=[0.3, 2.5]),
                                    legend=None),
        opacity=alt.Opacity("is_total:Q",
                            scale=alt.Scale(domain=[0, 1], range=[0.9, 1.0]),
                            legend=None),
        tooltip=[
            alt.Tooltip("Capitulo Descripcion:N", title="Capítulo"),
            alt.Tooltip("Nombre Proyecto:N", title="Proyecto"),
            alt.Tooltip("Porcentaje:Q", title="% Adicional", format=".2f")
        ],
    )
    .properties(width=950, height=650)
)

chart


### el mejor hasta ahora

In [73]:
# === 2) Re-plot with a continuous palette (0% = white, 100% = full color) ===
# A smooth multi-stop gradient: white → yellow-green → teal → blue (no black)
continuous_range = ["#ffffff", "#fde725", "#5ec962", "#21908d", "#3b528b"]

chart = (
    alt.Chart(crossmap_full)
    .mark_rect(stroke="black")
    .encode(
        x=alt.X("Nombre Proyecto:O", sort=cat_proj, title="Proyecto"),
        y=alt.Y("Capitulo Descripcion:O", sort=cat_cap, title="Capítulo"),
        color=alt.Color(
            "Porcentaje:Q",
            scale=alt.Scale(domain=[0, 100], range=continuous_range),
            legend=alt.Legend(title="% Adicional (0 = blanco, 100 = máximo)")
        ),
        strokeWidth=alt.StrokeWidth("is_total:Q", scale=alt.Scale(domain=[0,1], range=[0.3,2.5]), legend=None),
        opacity=alt.Opacity("is_total:Q", scale=alt.Scale(domain=[0,1], range=[0.9,1.0]), legend=None),
        tooltip=[
            alt.Tooltip("Capitulo Descripcion:N", title="Capítulo"),
            alt.Tooltip("Nombre Proyecto:N", title="Proyecto"),
            alt.Tooltip("Porcentaje:Q", title="% Adicional", format=".2f"),
        ],
    )
    .properties(width=950, height=650)
)

import altair as alt
alt.data_transformers.enable("vegafusion")  # allows larger dataframes

chart


alt.Chart(...)

### corregir los totales

In [76]:
# Normalize
df["Tipo Pedido"] = df["Tipo Pedido"].str.strip().str.upper()

# --- per (capítulo, proyecto): distinct pedidos ---
tot_pairs = (
    df.groupby(["SkIdCapitulo", "SkIdProyecto"])["SkIdPedido"]
      .nunique()
      .rename("Total")
)
adic_pairs = (
    df[df["Tipo Pedido"] == "ADICIONAL"]
      .groupby(["SkIdCapitulo", "SkIdProyecto"])["SkIdPedido"]
      .nunique()
      .rename("Adic")
)

pairs = (
    pd.concat([tot_pairs, adic_pairs], axis=1)
      .fillna(0)
      .reset_index()
)
pairs["Porcentaje"] = (pairs["Adic"] / pairs["Total"] * 100).round(2)

# Attach names
pairs = (
    pairs.merge(df_cap[["SkIdCapitulo", "Capitulo Descripcion"]].drop_duplicates(), on="SkIdCapitulo", how="left")
         .merge(df_proj[["SkIdProyecto", "Nombre Proyecto"]].drop_duplicates(), on="SkIdProyecto", how="left")
)

# --- row totals (by capítulo): distinct pedidos across all proyectos ---
tot_row = df.groupby("SkIdCapitulo")["SkIdPedido"].nunique().rename("Total")
adic_row = df[df["Tipo Pedido"] == "ADICIONAL"].groupby("SkIdCapitulo")["SkIdPedido"].nunique().rename("Adic")
row_totals = (
    pd.concat([tot_row, adic_row], axis=1).fillna(0).reset_index()
      .merge(df_cap[["SkIdCapitulo", "Capitulo Descripcion"]].drop_duplicates(), on="SkIdCapitulo", how="left")
      .assign(Porcentaje=lambda d: (d["Adic"] / d["Total"] * 100).round(2), **{"Nombre Proyecto": "TOTAL"})
      .loc[:, ["Capitulo Descripcion", "Nombre Proyecto", "Porcentaje"]]
)

# --- column totals (by proyecto): distinct pedidos across all capítulos ---
tot_col = df.groupby("SkIdProyecto")["SkIdPedido"].nunique().rename("Total")
adic_col = df[df["Tipo Pedido"] == "ADICIONAL"].groupby("SkIdProyecto")["SkIdPedido"].nunique().rename("Adic")
col_totals = (
    pd.concat([tot_col, adic_col], axis=1).fillna(0).reset_index()
      .merge(df_proj[["SkIdProyecto", "Nombre Proyecto"]].drop_duplicates(), on="SkIdProyecto", how="left")
      .assign(Porcentaje=lambda d: (d["Adic"] / d["Total"] * 100).round(2), **{"Capitulo Descripcion": "TOTAL"})
      .loc[:, ["Capitulo Descripcion", "Nombre Proyecto", "Porcentaje"]]
)

# --- assemble crossmap_full for plotting (continuous palette step you already have) ---
crossmap_full = pd.concat([
    pairs.loc[:, ["Capitulo Descripcion", "Nombre Proyecto", "Porcentaje"]],
    row_totals,
    col_totals
], ignore_index=True)

crossmap_full["is_total"] = ((crossmap_full["Capitulo Descripcion"] == "TOTAL") |
                             (crossmap_full["Nombre Proyecto"] == "TOTAL")).astype(int)

print("[DEBUG] check pairs sample:\n", pairs.head().to_string(index=False))
print("[DEBUG] row totals sample:\n", row_totals.head().to_string(index=False))
print("[DEBUG] col totals sample:\n", col_totals.head().to_string(index=False))


[DEBUG] check pairs sample:
  SkIdCapitulo  SkIdProyecto  Total  Adic  Porcentaje          Capitulo Descripcion                           Nombre Proyecto
      1006166          1006      1   1.0       100.0 INSTALACIONES HIDROSANITARIAS                 Parque Engativá - Etapa I
      1009342          1009      1   1.0       100.0              GASTOS GENERALES                   TRIBECA - Etapas I y II
     10021454         10021      5   5.0       100.0                    POSTVENTAS                  PARQUE ENGATIVA ETAPA II
     10030986         10030      1   1.0       100.0                   IMPREVISTOS Caminos de Sie - Urb y Zonas Comunes MZ 2
    100291591         10029      1   1.0       100.0                VIAS Y ANDENES Caminos de Sie - Urb Y Zonas Comunes MZ 3
[DEBUG] row totals sample:
          Capitulo Descripcion Nombre Proyecto  Porcentaje
INSTALACIONES HIDROSANITARIAS           TOTAL       100.0
             GASTOS GENERALES           TOTAL       100.0
                   

In [77]:
# === 2) Re-plot with a continuous palette (0% = white, 100% = full color) ===
# A smooth multi-stop gradient: white → yellow-green → teal → blue (no black)
continuous_range = ["#ffffff", "#fde725", "#5ec962", "#21908d", "#3b528b"]

chart = (
    alt.Chart(crossmap_full)
    .mark_rect(stroke="black")
    .encode(
        x=alt.X("Nombre Proyecto:O", sort=cat_proj, title="Proyecto"),
        y=alt.Y("Capitulo Descripcion:O", sort=cat_cap, title="Capítulo"),
        color=alt.Color(
            "Porcentaje:Q",
            scale=alt.Scale(domain=[0, 100], range=continuous_range),
            legend=alt.Legend(title="% Adicional (0 = blanco, 100 = máximo)")
        ),
        strokeWidth=alt.StrokeWidth("is_total:Q", scale=alt.Scale(domain=[0,1], range=[0.3,2.5]), legend=None),
        opacity=alt.Opacity("is_total:Q", scale=alt.Scale(domain=[0,1], range=[0.9,1.0]), legend=None),
        tooltip=[
            alt.Tooltip("Capitulo Descripcion:N", title="Capítulo"),
            alt.Tooltip("Nombre Proyecto:N", title="Proyecto"),
            alt.Tooltip("Porcentaje:Q", title="% Adicional", format=".2f"),
        ],
    )
    .properties(width=950, height=650)
)

import altair as alt
alt.data_transformers.enable("vegafusion")  # allows larger dataframes

chart


alt.Chart(...)

In [52]:
# --- Normalize case and compute % ADICIONAL ---
df["Tipo Pedido"] = df["Tipo Pedido"].str.strip().str.upper()

crossmap = (
    df.groupby(["SkIdCapitulo", "SkIdProyecto"])["Tipo Pedido"]
      .apply(lambda s: (s == "ADICIONAL").mean() * 100)
      .reset_index(name="Porcentaje")
)

# --- Inspect distribution to guide color ranges ---
s = crossmap["Porcentaje"]
print("[DEBUG] Non-zero share:", round(100*(s>0).mean(),2), "%")
print(s.describe(percentiles=[.1,.25,.5,.75,.9,.95,.99]).round(2).to_string())

# --- Suggest bin edges and color ideas ---
q = s[s>0].quantile([0,.25,.5,.75,1]).round(2).tolist()
print(f"\n[DEBUG] Suggested bins (quartiles of non-zero %): {q}")
print("\nRecommended palette plan:")
print(" • Crossmap cells → sequential from pale orange (#fee6ce) to deep red (#a63603)")
print(" • Row/column totals → blues (light #c6dbef → dark #08306b) to contrast")
print(" • 0% (no Adicional) stays white for clarity")


[DEBUG] Non-zero share: 94.97 %
count    596.00
mean      82.12
std       29.52
min        0.00
10%       33.15
25%       75.67
50%      100.00
75%      100.00
90%      100.00
95%      100.00
99%      100.00
max      100.00

[DEBUG] Suggested bins (quartiles of non-zero %): [4.17, 81.52, 100.0, 100.0, 100.0]

Recommended palette plan:
 • Crossmap cells → sequential from pale orange (#fee6ce) to deep red (#a63603)
 • Row/column totals → blues (light #c6dbef → dark #08306b) to contrast
 • 0% (no Adicional) stays white for clarity


## v2

In [7]:
import pandas as pd
from pathlib import Path

# === Load tables ===
carpeta = Path("20251003/20251003")
df_spec = pd.read_csv(carpeta / "ADP_DTM_DIM.EspecicficacionDePedidos.csv")     # has 'Tipo Pedido'
df_ped  = pd.read_csv(carpeta / "ADP_DTM_FACT.Pedidos.csv")                     # has 'SkIdProyecto'
df_proj = pd.read_csv(carpeta / "ADP_DTM_DIM.Proyecto.csv")                     # has 'SkIdProyecto'
print("[DEBUG] Tables loaded.")

# === Merge Tipo Pedido into Pedidos ===
merged = df_ped.merge(
    df_spec[["SkIdEmpresa", "SkIdPedido", "Tipo Pedido"]],
    on=["SkIdEmpresa", "SkIdPedido"],
    how="left"
)
print(f"[DEBUG] Total Pedidos: {len(merged):,}")

# === Count projects measured by Tipo Pedido ===
measured_projects = merged.loc[merged["Tipo Pedido"].notna(), "SkIdProyecto"].nunique()

# === True total: all projects in Proyecto table ===
total_projects = df_proj["SkIdProyecto"].nunique()

percentage_measured = round(measured_projects / total_projects * 100, 2)

print(f"[DEBUG] Projects with Tipo Pedido: {measured_projects}/{total_projects} ({percentage_measured}%)")


[DEBUG] Tables loaded.
[DEBUG] Total Pedidos: 99,836
[DEBUG] Projects with Tipo Pedido: 59/85 (69.41%)


## Debug

In [50]:
# === 1) Quick stats to see what's going on ===
s = crossmap["Porcentaje"]  # % Adicional per (Capítulo, Proyecto)
print("[DEBUG] Cells:", s.size, "| Non-zero %:", round(100*(s>0).mean(), 2))
print("[DEBUG] Describe:\n", s.describe(percentiles=[.1,.25,.5,.75,.9,.95,.99]).round(2).to_string())
print("[DEBUG] Top non-zero values:\n", s[s>0].sort_values(ascending=False).head(10).to_string(index=False))

# === 2) Build 4 quantile bins (balanced, no white) ===
q_edges = [0.0, s.quantile(.50), s.quantile(.80), s.quantile(.95), s.max()]  # median, 80th, 95th
edges = sorted({round(float(x), 2) for x in q_edges})
if len(edges) < 5:  # fallback if too many zeros collapse quantiles
    edges = [0.0, 1.0, 5.0, 20.0, 100.0]
labels = [f"{edges[i]}–{edges[i+1]}" for i in range(len(edges)-1)]
print("[DEBUG] Bin edges:", edges)




[DEBUG] Cells: 596 | Non-zero %: 0.0
[DEBUG] Describe:
 count    596.0
mean       0.0
std        0.0
min        0.0
10%        0.0
25%        0.0
50%        0.0
75%        0.0
90%        0.0
95%        0.0
99%        0.0
max        0.0
[DEBUG] Top non-zero values:
 Series([], )
[DEBUG] Bin edges: [0.0, 1.0, 5.0, 20.0, 100.0]


In [51]:
print("[DEBUG] Unique values in df['Tipo Pedido']:", df["Tipo Pedido"].dropna().unique()[:20])
print("[DEBUG] Counts per type:")
print(df["Tipo Pedido"].value_counts(dropna=False))

print("\n[DEBUG] Counts of Tipo Pedido per Capítulo-Proyecto combination:")
print(df.groupby(["SkIdCapitulo", "SkIdProyecto"])["Tipo Pedido"].nunique().describe())

print("\n[DEBUG] Sample of Capítulo-Proyecto pairs and their 'Tipo Pedido' values:")
sample = (
    df.groupby(["SkIdCapitulo", "SkIdProyecto"])["Tipo Pedido"]
      .apply(lambda s: s.unique()[:5])
      .sample(10, random_state=1)
)
print(sample.to_string())

print("\n[DEBUG] How many rows actually match 'Adicional':", (df["Tipo Pedido"] == "Adicional").sum())


[DEBUG] Unique values in df['Tipo Pedido']: ['PRESUPUESTAL' 'ADICIONAL']
[DEBUG] Counts per type:
Tipo Pedido
ADICIONAL       67609
PRESUPUESTAL    32227
Name: count, dtype: int64

[DEBUG] Counts of Tipo Pedido per Capítulo-Proyecto combination:
count    596.000000
mean       1.441275
std        0.496957
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max        2.000000
Name: Tipo Pedido, dtype: float64

[DEBUG] Sample of Capítulo-Proyecto pairs and their 'Tipo Pedido' values:
SkIdCapitulo  SkIdProyecto
1002112842    100211          [ADICIONAL, PRESUPUESTAL]
1002152863    100215                        [ADICIONAL]
1001842509    100184                        [ADICIONAL]
1002423550    100242          [ADICIONAL, PRESUPUESTAL]
1002012895    100201                        [ADICIONAL]
100312190     10031           [PRESUPUESTAL, ADICIONAL]
1002774017    100277                     [PRESUPUESTAL]
1002042714    100204                        [ADICIONAL]
1001352321